Mithilfe dieses Notebooks wird die Quelldatei mit Bibtex-Code als Dataframe eingelesen und gemäß den Vorgaben den Plain-Stils in einen Literaturstring konvertiert. Die Ausgabe ist eine csv-Datei mit den vier Spalten Literaturstring | Stil | Literaturtyp | Bibtex.

In [1]:
import pandas as pd

# Pfad zur .bib-Datei mit article-, proceedings- und inproceedings-Einträgen
bib_file = 'anthology.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries = []

# Bibtex-Einträge extrahieren
with open(bib_file, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries.append(entry)

# DataFrame erstellen
df = pd.DataFrame({'BibTeX': bib_entries})


# Pfad zur .bib-Datei mit incollection-Einträgen
bib_file2 = '1016incollection_reviewed.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries2 = []

# Bibtex-Einträge extrahieren
with open(bib_file2, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries2.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries2.append(entry)

# DataFrame erstellen
df5 = pd.DataFrame({'BibTeX': bib_entries2})


# Pfad zur .bib-Datei mit book-Einträgen
bib_file3 = '1000books.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries3 = []

# Bibtex-Einträge extrahieren
with open(bib_file3, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries3.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries3.append(entry)

# DataFrame erstellen
df6 = pd.DataFrame({'BibTeX': bib_entries3})


# Pfad zur .bib-Datei mit book-Einträgen
bib_file4 = '1000PhD.bib'

# Liste zum Speichern der Bibtex-Einträge
bib_entries4 = []

# Bibtex-Einträge extrahieren
with open(bib_file4, 'r', encoding='utf-8') as file:
    entry = ''
    for line in file:
        # Einträge erkennen
        if line.strip().startswith('@'):
            if entry:
                bib_entries4.append(entry)
            entry = line
        else:
            entry += line.rstrip() + '\n'
    # Letzten Eintrag hinzufügen
    if entry:
        bib_entries4.append(entry)

# DataFrame erstellen
df7 = pd.DataFrame({'BibTeX': bib_entries4})

# DataFrame anzeigen
# display(df)

In [2]:
# Kürzung zu Testzwecken
#df = df.head(50000)

In [3]:
# Anpassung des Plain-Styles in der Bibliothek Pybtex

import re
import io
from pybtex.database import parse_string
from pybtex.database.input import bibtex
from pybtex.plugin import find_plugin
from pybtex.style.formatting import BaseStyle, toplevel
from pybtex.style.formatting.plain import Style as PlainStyle
from pybtex.style.template import (
    field, first_of, href, join, optional, optional_field, sentence, tag,
    together, words, node, FieldIsMissing
)
from pybtex import richtext
from pybtex.richtext import Text, Symbol


# Benutzerspezifischer Stil, der von PlainStyle erbt
class CustomPlainStyle(PlainStyle):
    
    def get_inproceedings_template(self, e):        
        # Eigene Formatierung für inproceedings: Vor den Seitenzahlen soll "pages" als Wort erscheinen, nach den Seitenzahlen ein Komma statt Punkt.
        template = toplevel [
            sentence [self.format_names('author')],
            self.format_title(e, 'title'),
            words [
                'In',
                sentence [
                    optional[ self.format_editor(e, as_sentence=False) ],
                    self.format_btitle(e, 'booktitle', as_sentence=False),
                    self.format_volume_and_series(e, as_sentence=False),
                    optional[ self.format_chapter_and_pages(e) ],
                    self.format_address_organization_publisher_date(e),
                ],
            ],
            sentence [ optional_field('note') ],
            self.format_web_refs(e),
        ]
        return template

    # Überschreibe die Methode, um nach der Jahreszahl einen Punkt zu erzeugen, kein Komma.
    def get_inbook_template(self, e):
        template = toplevel [
            self.format_author_or_editor(e),
            sentence [
                self.format_btitle(e, 'title', as_sentence=False),
                self.format_chapter_and_pages(e),
            ],
            self.format_volume_and_series(e),
            sentence [
                field('publisher'),
                optional_field('address'),
                optional [
                    words [field('edition'), 'edition']
                ],
                field('year'), 
            ],
            optional_field('note'),
            self.format_web_refs(e),
        ]
        return template

    # Überschreibe die Methode, um nach der Jahreszahl einen Punkt zu erzeugen, kein Komma.
    def get_booklet_template(self, e):
        template = toplevel [
            self.format_names('author'),
            self.format_title(e, 'title'),
            sentence [
                optional_field('howpublished'),
                optional_field('address'),
                field('year'),
            ],
            optional_field('note'),
            self.format_web_refs(e),
        ]
        return template

In [4]:
# Den benutzerdefinierten Formatter finden
plain_style = CustomPlainStyle()
plain_backend = find_plugin('pybtex.backends', 'text')()

# Funktion, die Bibtex-Code in einen Literaturstring umwandelt
def umwandler(bibtex_code):
    
        # BibTeX-Code parsen
        bib_data = parse_string(bibtex_code, 'bibtex')

        # Ändern des Eintragstyps in Semantic Scholar von 'inproceedings' zu 'proceedings', von 'masterthesis' zu 'phdthesis',
        # weil dort booktitle fehlt. Allgemein ist in SemScholar die Datenquali sehr schlecht
        # von 'conference' zu 'inproceedings'
        for entry_key in list(bib_data.entries.keys()):
            entry = bib_data.entries[entry_key]
            #if entry.type.lower() == 'inproceedings':
                #entry.type = 'manual'
            if entry.type.lower() == 'conference':
                entry.type = 'inproceedings'
            if entry.type.lower() == 'masterthesis':
                entry.type = 'phdthesis'
            if entry.type == 'InCollection':
                entry.type = 'incollection'
        
        # Einträge formatieren
        formatted_entries = plain_style.format_entries(bib_data.entries.values())
            
        # Custom Backend zum Erfassen der Ausgabe in einem String
        class StringBackend(plain_backend.__class__):
            def __init__(self):
                super().__init__()
                self.output = io.StringIO()
        
            def write_entry(self, label, text):
                self.output.write(text + "\n")
        
            def get_output(self):
                return self.output.getvalue()

            def format_url(self, url):
                return Text(url)
        
        # Custom Backend verwenden
        string_backend = StringBackend()
        try:
            for entry in formatted_entries:
                string_backend.write_entry(entry.key, entry.text.render(string_backend))
        except FieldIsMissing as ex:
            return ex

        # Formatierten String erhalten
        formatted_string = string_backend.get_output()

        # Nachbearbeitung: \url{}-Kommando entfernen
        formatted_string = formatted_string.replace('\\url ', '')

        return formatted_string

# Funktion, die den Literaturtyp aus dem Bibtex-Code ausliest
def entrytype(bibtex_code):
    # BibTeX-Code parsen
    bib_data = parse_string(bibtex_code, 'bibtex')

    #Extrahiere Eintragstyp
    entry = list(bib_data.entries.values())[0]
    return entry.type



In [5]:
#Erzeugung der Spalte mit den Literaturtypen
df["Literaturtyp"] = df["BibTeX"].apply(entrytype)
df5["Literaturtyp"] = df5["BibTeX"].apply(entrytype)
df6["Literaturtyp"] = df6["BibTeX"].apply(entrytype)
df7["Literaturtyp"] = df7["BibTeX"].apply(entrytype)
#display(df)

In [6]:
#Filtern nach einem Literaturtyp und Beschränkung der Einträge auf 700 für Trainingsdaten und 300 für die Testdaten
df2 = df[df["Literaturtyp"] == 'article']
#df2 = df2.head(700)
df2 = df2.iloc[700:1000]

df3 = df[df["Literaturtyp"] == 'proceedings']
#df3 = df3.head(700)
df3 = df3.iloc[700:1000]

df4 = df[df["Literaturtyp"] == 'inproceedings']
#df4 = df4.head(700)
df4 = df4.iloc[700:1000]

#df5 = df5.head(700)
df5 = df5.iloc[700:1000]
#df6 = df6.head(700)
df6 = df6.iloc[700:1000]
#df7 = df7.head(700)
df7 = df7.iloc[700:1000]

#Vereinigung zu einem Dataframe
df_plain = pd.concat([df2,df3,df4,df5,df6,df7], ignore_index=True)

display(df_plain.head(5))
display(df_plain.info())

,BibTeX,Literaturtyp
0,"@article{mansouri-bigvand-etal-2017-joint,\n ...",article
1,"@article{zhang-etal-2017-aspect,\n title = ...",article
2,"@article{gallagher-etal-2017-anchored,\n ti...",article
3,"@article{joty-etal-2017-discourse,\n title ...",article
4,"@article{rozovskaya-etal-2017-adapting,\n t...",article


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BibTeX        1800 non-null   object
 1   Literaturtyp  1800 non-null   object
dtypes: object(2)
memory usage: 28.3+ KB


None

In [7]:
#Erzeugung der Spalte mit den Literaturstrings
df_plain["Referenzstring"] = df_plain["BibTeX"].apply(umwandler)

display(df_plain.head(5))

,BibTeX,Literaturtyp,Referenzstring
0,"@article{mansouri-bigvand-etal-2017-joint,\n ...",article,"Anahita Mansouri Bigvand, Te Bu, and Anoop Sar..."
1,"@article{zhang-etal-2017-aspect,\n title = ...",article,"Yuan Zhang, Regina Barzilay, and Tommi Jaakkol..."
2,"@article{gallagher-etal-2017-anchored,\n ti...",article,"Ryan J. Gallagher, Kyle Reing, David Kale, and..."
3,"@article{joty-etal-2017-discourse,\n title ...",article,"Shafiq Joty, Francisco Guzmán, Lluís Màrquez, ..."
4,"@article{rozovskaya-etal-2017-adapting,\n t...",article,"Alla Rozovskaya, Dan Roth, and Mark Sammons. A..."


In [9]:
ind = 500
print(df_plain["Referenzstring"][ind])
print(df_plain["BibTeX"][ind])
print(df_plain["Literaturtyp"][ind])

Malvina Nissim, Viviana Patti, Barbara Plank, and Claudia Wagner, editors. Proceedings of the Second Workshop on Computational Modeling of People's Opinions, Personality, and Emotions in Social Media, New Orleans, Louisiana, USA, June 2018. Association for Computational Linguistics. URL: https://aclanthology.org/W18-1100, doi:10.18653/v1/W18-11.

@proceedings{ws-2018-modeling,
    title = "Proceedings of the Second Workshop on Computational Modeling of People{'}s Opinions, Personality, and Emotions in Social Media",
    editor = "Nissim, Malvina  and
      Patti, Viviana  and
      Plank, Barbara  and
      Wagner, Claudia",
    month = jun,
    year = "2018",
    address = "New Orleans, Louisiana, USA",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/W18-1100",
    doi = "10.18653/v1/W18-11",
}

proceedings


In [10]:
# Suche nach Strings, die mit "missing" beginnen; aus irgendeinem Grund muss der Referenzstring als String behandelt werden
result = df_plain[df_plain['Referenzstring'].astype(str).str.startswith('missing')]

# Ergebnis anzeigen
display(result)

,BibTeX,Literaturtyp,Referenzstring
10,"@article{nn-2017-reviewers,\n title = ""Revi...",article,missing author in nn-2017-reviewers
99,"@article{nn-2016-reviewers,\n title = ""Revi...",article,missing author in nn-2016-reviewers
284,"@article{nn-2014-erratum,\n title = ""Erratu...",article,missing author in nn-2014-erratum
1026,"@incollection {rHEL98bx,\n author = {Helleka...",incollection,missing booktitle in rHEL98bx
1029,"@incollection{rHOW07a,\n author = {L. How...",incollection,missing booktitle in rHOW07a
1031,"@incollection {rLAR98ax,\n AUTHOR =""G. La...",incollection,missing year in rLAR98ax
1062,"@incollection {rNIE98ax,\n AUTHOR =""H. Ni...",incollection,missing year in rNIE98ax
1128,"@INCOLLECTION {vHIC98cx,\n AUTHOR =""F. J....",incollection,missing year in vHIC98cx
1129,"@Incollection {vHIC00bx,\n AUTHOR =""F. J. ...",incollection,missing year in vHIC00bx
1132,"@InCollection{Andre2018,\n author = {E...",incollection,missing booktitle in Andre2018


In [11]:
# Filtern und Zeilen löschen, die mit "missing" beginnen
df_plain = df_plain[~df_plain['Referenzstring'].astype(str).str.startswith('missing')]

# Suche nach Strings, die mit "missing" beginnen; aus irgendeinem Grund muss der Referenzstring als String behandelt werden
result = df_plain[df_plain['Referenzstring'].astype(str).str.startswith('missing')]

# Ergebnis anzeigen
display(result)

,BibTeX,Literaturtyp,Referenzstring


In [12]:
#Erzeugung der Spalte mit dem Stil
df_plain.insert(1, 'Style', 'plain')

# Neuordnung der Spalten
new_order = ['Referenzstring', 'Style', 'Literaturtyp', 'BibTeX']
df_plain = df_plain[new_order]
display(df_plain)

# DataFrame als CSV-Datei mit Pipe-Separator exportieren
#df_plain.to_csv('training_plain.csv', sep='|', index=False, encoding='utf-8')
df_plain.to_csv('test_plain.csv', sep='|', index=False, encoding='utf-8')

,Referenzstring,Style,Literaturtyp,BibTeX
0,"Anahita Mansouri Bigvand, Te Bu, and Anoop Sar...",plain,article,"@article{mansouri-bigvand-etal-2017-joint,\n ..."
1,"Yuan Zhang, Regina Barzilay, and Tommi Jaakkol...",plain,article,"@article{zhang-etal-2017-aspect,\n title = ..."
2,"Ryan J. Gallagher, Kyle Reing, David Kale, and...",plain,article,"@article{gallagher-etal-2017-anchored,\n ti..."
3,"Shafiq Joty, Francisco Guzmán, Lluís Màrquez, ...",plain,article,"@article{joty-etal-2017-discourse,\n title ..."
4,"Alla Rozovskaya, Dan Roth, and Mark Sammons. A...",plain,article,"@article{rozovskaya-etal-2017-adapting,\n t..."
...,...,...,...,...
1795,Mélodie Greenlees. Intelligente technische Sys...,plain,phdthesis,"@phdthesis{Greenlees.2012,\n author = {Greenle..."
1796,Personnalisée Greenly. Database techniques for...,plain,phdthesis,"@phdthesis{Greenly.2013,\n author = {Greenly, ..."
1797,Garçon Greeve. Anforderungsbasierte Modellieru...,plain,phdthesis,"@phdthesis{Greeve.2018,\n author = {Greeve, Ga..."
1798,Ruò Glasscoo. Temporal models of streaming soc...,plain,phdthesis,"@phdthesis{Glasscoo.2014,\n author = {Glasscoo..."
